## Rectangle Tutorial

Welcome to the tutorial on how to draw a rectangle using the new QtGui based OpenGL api of PySide2.

As we have mentioned on the first tutorial on how to draw the `Hello Triangle`. I will be giving only the relative parts of the code and explain the differences rather than commenting everything.

Let's see the final output of our application

In [2]:
import subprocess

subprocess.run(["python", "app.py"])

CompletedProcess(args=['python', 'app.py'], returncode=0)

Not so bad right !

I am skipping the code on the window holding the gl widget. 

Here is the constructor of our gl widget.

In [ ]:
class RectangleGL(QOpenGLWidget):
    "Texture loading opengl widget"

    def __init__(self, parent=None):
        "Constructor"
        QOpenGLWidget.__init__(self, parent)
        tutoTutoDir = os.path.dirname(__file__)
        tutoPardir = os.path.join(tutoTutoDir, os.pardir)
        tutoPardir = os.path.realpath(tutoPardir)
        mediaDir = os.path.join(tutoPardir, "media")
        shaderDir = os.path.join(mediaDir, "shaders")
        #
        availableShaders = ["rectangle", "triangle"]
        self.shaders = {
            name: {
                "fragment": os.path.join(shaderDir, name + ".frag"),
                "vertex": os.path.join(shaderDir, name + ".vert")
            } for name in availableShaders
        }
        self.core = "--coreprofile" in QCoreApplication.arguments()

        # opengl data related
        self.context = QOpenGLContext()
        self.program = QOpenGLShaderProgram()
        self.vao = QOpenGLVertexArrayObject()
        self.vbo = QOpenGLBuffer(QOpenGLBuffer.VertexBuffer)
        
        # ############ Diff 1 ##########################
        # this is the novelty in this code. We specify #
        # indices of triangles that would made up      #
        # our rectangle. Notice its data type          #
        self.indices = np.array([
            0, 1, 3,  # first triangle
            1, 2, 3  # second triangle
        ], dtype=ctypes.c_uint)

        # vertex data of the panel that would hold the image
        
        self.vertexData = np.array([
            # corners of the rectangle
            0.5,  0.5,  0.0,  # top right
            0.5,  -0.5, 0.0,  # bottom right
            -0.5, -0.5, 0.0,  # bottom left
            -0.5, 0.5,  0.0,  # top left
        ], dtype=ctypes.c_float)

        self.rectColor = QVector4D(0.0, 1.0, 1.0, 0.0)

As you can see it is fairly close to the constructor of the triangle widget.
The initialization is exactly the same as triangle widget so we are skipping that entirely.

Let's see the drawing function.

In [ ]:
 def paintGL(self):
        "paint gl"
        funcs = self.context.functions()
        # clean up what was drawn
        funcs.glClear(pygl.GL_COLOR_BUFFER_BIT)

        # bind texture
        vaoBinder = QOpenGLVertexArrayObject.Binder(self.vao)
        self.program.bind()

        # draw stuff
        ########## Diff #############
        # This is another drawing function in opengl
        # Notice that its signature is from OpenGL ES 2
        
        funcs.glDrawElements(
            pygl.GL_TRIANGLES,
            self.indices.size,
            pygl.GL_UNSIGNED_INT,
            self.indices.tobytes())
            # VoidPtr(self.indices.tobytes() * ctypes.sizeof(ctypes.c_uint)))
        vaoBinder = None
        self.program.release()

- The first parameter is the drawing mode.
- The second is the number of elements that is to be drawn, since the indice specifies the index of an element that is to be drawn, it is equal to the size of indices.
- We specify the type of the elements of indices
- We specify the data of indices. 

If we want to follow documentation more closely we can also use the commented line instead of the final parameter which creates the void pointer to data of the indices

That's it now you know how to draw a rectangle in opengl api of pyside2